In [12]:
!pip install -q gurobipy

In [13]:
!pip install -q gurobipy

import os

os.environ['GRB_WLSACCESSID'] = 'YOUR_ACCESS_ID'
os.environ['GRB_WLSSECRET'] = 'YOUR_SECRET_KEY'
os.environ['GRB_LICENSEID'] = '2718149'  # Your license ID

import gurobipy as gp
from gurobipy import GRB

m = gp.Model('colab_test')
x = m.addVar(name='x')
m.setObjective(x, GRB.MAXIMIZE)
m.addConstr(x <= 5)
m.optimize()


Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 1 columns and 1 nonzeros (Max)
Model fingerprint: 0xcd21b9b4
Model has 1 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e+00


In [14]:
# --- Activate Gurobi Academic (WLS) in Colab ---
!pip install -q gurobipy

# Create the license file in Colab (ephemeral each session)
license_text = """# Gurobi WLS license file
# Your credentials are private and should not be shared or copied to public repositories.
# Visit https://license.gurobi.com/manager/doc/overview for more information.
WLSACCESSID=3b0100a9-3166-4003-8b5a-dd4c80226c63
WLSSECRET=f7be1a22-8281-423f-ab1d-1244842a6ad6
LICENSEID=2718149
"""
with open('/root/gurobi.lic', 'w') as f:
    f.write(license_text)

import os
os.environ['GRB_LICENSE_FILE'] = '/root/gurobi.lic'

# Assignment 3: Nonlinear Programming

## Instructions:
1. Answer the questions in this Jupyter Notebook file and upload it in the Assignment 3 entry under the Assignments tab in MyCourses. Answer each question in its place. Late submissions are NOT accepted. Also, files not in the correct format (other than a Jupyter Notebook) are NOT accepted.
2. You can submit your answer file multiple times, but only the last submission is kept and graded.
3. For each question, print all the results that the question asks. Also, prevent printing unnecessary information.
4. Your submissions will be graded based on the correctness of the results.
5. For formulation problems, the only acceptable answers are the correct formulas. If you are using an additional variable, introduce it clearly.
6. **Unless otherwise stated, whenever we mention the "risk," we mean the standard deviation and not the variance.**

In [15]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *

# Problem 1: The Markowitz Problem (50 points)

## The Story:

A rational investor wishes for a low variance on return and a high expected rate of return. However, the two goals seldom align. For example, you can always keep variance down by investing in bonds over stocks, but you do so at the expense of a decent rate of return. So, an investor's optimal portfolio could be described by ($r, \sigma$), where $r$ is a desired (and feasible) average rate of return, and $\sigma$ is the minimal standard deviation possible for this given $r$. (Put differently, an investor might wish to find the highest rate of return possible for a given acceptable level of risk.) This problem is due originally to Harry Markowitz in the 1950's.

## The Data:

The data in the Excel file "Markowitz" provides the performances of five stock exchange indices from 31 Dec 2010 to 1 June 2016. Throughout this problem, we will use that dataset to find the optimal portfolio. But before that, we go through some warm-up questions.

### Caution:

In this problem, do not convert the values to percentages. Using percentages can lead to wrong results if you're not cautious.

## Question 1: (5 points)
Use the following code to calculate the monthly rate of return (RoR) of each index.

What does *pct_change(-1)* in the following code do?

In [16]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row. (The earliest date doesn't have any data before to be compared with.)
rate_of_return_df.dropna(inplace = True)

display(rate_of_return_df)

,FTSE 100,DAX,DJIA,DJ Asian Titans 50,Russell 2000
2016-06-01,-0.042370,-0.063923,-0.008266,-0.016583,-0.004754
2016-05-03,-0.001778,0.022290,0.000763,-0.013982,0.021170
2016-04-01,0.010850,0.007371,0.005007,0.017413,0.015098
2016-03-01,0.012760,0.049509,0.070753,0.080918,0.077503
2016-02-01,0.002186,-0.030895,0.003049,-0.039002,-0.001429
...,...,...,...,...,...
2011-05-02,-0.013163,-0.029379,-0.018793,-0.026757,-0.019635
2011-04-01,0.027264,0.067196,0.039839,0.015772,0.025772
2011-03-01,-0.014214,-0.031766,0.007638,-0.047549,0.024409
2011-02-01,0.022361,0.027530,0.028121,0.030477,0.054016


*Your answer here*


## Question 1
**Answer (for the markdown cell after Q1):**

`pct_change(-1)` computes the percentage change **from the next row to the current row**.

For each date $ t $, it calculates

$$
\frac{\text{Price}_t - \text{Price}_{t+1}}{\text{Price}_{t+1}}
$$

so each entry is the **monthly rate of return going from the following month back to the current month** (because of the `-1` shift).

## Question 2: (10 points)

Estimate and return **average, variance, and standard deviation of the RoR for each index in that period**. Then, estimate and return **the covariance and correlation matrix between each pair of indices in that period**. (Hint: You can use the *mean()*, *var()*, *cov()*, and *corr()* functions from the Pandas package.)

Hint: If done correctly, you must have:
1. The average monthly return of Russell 2000 is 0.0069.
2. The covariance between DAX and DJIA is 0.001224.

In [19]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row. (The earliest date doesn't have any data before to be compared with.)
rate_of_return_df.dropna(inplace = True)

#############################################
# Your code here
# Average monthly return for each index
RoR_Avg = rate_of_return_df.mean()

# Variance of monthly returns for each index
RoR_Var = rate_of_return_df.var()

# Standard deviation (risk) for each index
RoR_Std = rate_of_return_df.std()

# Correlation matrix between indices
RoR_Corr = rate_of_return_df.corr()

# Print results clearly
print("Average monthly returns:")
print(RoR_Avg)
print("\nVariances of monthly returns:")
print(RoR_Var)
print("\nStandard deviations (risks) of monthly returns:")
print(RoR_Std)
print("\nCorrelation matrix:")
print(RoR_Corr)



Average monthly returns:
FTSE 100              0.000695
DAX                   0.006402
DJIA                  0.006931
DJ Asian Titans 50   -0.001075
Russell 2000          0.006875
dtype: float64

Variances of monthly returns:
FTSE 100              0.001059
DAX                   0.002794
DJIA                  0.001082
DJ Asian Titans 50    0.002045
Russell 2000          0.002147
dtype: float64

Standard deviations (risks) of monthly returns:
FTSE 100              0.032549
DAX                   0.052861
DJIA                  0.032890
DJ Asian Titans 50    0.045223
Russell 2000          0.046334
dtype: float64

Correlation matrix:
                    FTSE 100       DAX      DJIA  DJ Asian Titans 50  \
FTSE 100            1.000000  0.755267  0.772833            0.687467   
DAX                 0.755267  1.000000  0.703816            0.668050   
DJIA                0.772833  0.703816  1.000000            0.721454   
DJ Asian Titans 50  0.687467  0.668050  0.721454            1.000000   
Russ

## Question 3: (10 points)

Formulate an NLP to devise the optimal portfolio that minimizes the portfolio risk (the standard deviation) subject to non-negative expected return. In your formulation, remember to include the constraint that the sum of the weights of the stocks in the portfolio should be equal to 1, and the weights should be non-negative.

What are **the optimal portfolio, the optimal risk, and the expected return of the optimal portfolio?**

Hint 1: This problem is an implementation of the "Risk Minimization Formulation" you have seen in the class. \
Hint 2: Minimizing the variance is equivalent to minimizing the standard deviation, as the variance is the square of the standard deviation, an increasing function. You may use this fact because minimizing variance is more straightforward to code than minimizing standard deviation. Nevertheless, report the standard deviation and not the variance in your answer.

In [20]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row.
rate_of_return_df.dropna(inplace = True)

# Calculate the average rate of return for each stock.
RoR_Avg = rate_of_return_df.mean()

# Calculate the covariance between each pair of stocks.
RoR_Cov = rate_of_return_df.cov()

###################################################################
# Create a new model for Question 3: risk minimization
# Your code here
model = gb.Model('Question3')
model.Params.LogToConsole = 0
m = len(Stock_Name)

# Decision variables: portfolio weights (one per index), nonnegative, sum to 1
x = model.addVars(m, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="x")

# Quadratic objective: minimize portfolio variance x' Σ x
var_expr = gb.QuadExpr()
for i in range(m):
    for j in range(m):
        var_expr.add(x[i] * RoR_Cov.iloc[i, j] * x[j])

model.setObjective(var_expr, GRB.MINIMIZE)

# Full investment: sum of weights = 1
model.addConstr(gb.quicksum(x[i] for i in range(m)) == 1.0, name="budget")

# Optimize
model.optimize()

# Extract optimal weights
opt_weights_q3 = [x[i].X for i in range(m)]

# Compute portfolio variance and risk (standard deviation)
import math
w_vec_q3 = np.array(opt_weights_q3)
Sigma = RoR_Cov.values
port_var_q3 = float(w_vec_q3 @ Sigma @ w_vec_q3)
port_risk_q3 = math.sqrt(port_var_q3)

# Compute portfolio expected return
mu = RoR_Avg.values
port_ret_q3 = float(mu @ w_vec_q3)

print("Question 3 – Optimal weights (risk minimization):")
for i in range(m):
    print(f"  {Stock_Name[i]}: {opt_weights_q3[i]:.6f}")

print(f"\nQuestion 3 – Optimal portfolio risk (std dev): {port_risk_q3:.6f}")
print(f"Question 3 – Expected return of optimal portfolio: {port_ret_q3:.66f}")


Set parameter LogToConsole to value 0
Question 3 – Optimal weights (risk minimization):
  FTSE 100: 0.522944
  DAX: 0.000000
  DJIA: 0.477056
  DJ Asian Titans 50: 0.000000
  Russell 2000: 0.000000

Question 3 – Optimal portfolio risk (std dev): 0.030801
Question 3 – Expected return of optimal portfolio: 0.003669783074765921998539042192533088382333517074584960937500000000


## Question 4: (10 points)

Model the problem of finding the optimal portfolio that maximizes the expected return subject to the constraint that the portfolio risk is less than or equal to 0.04. In your formulation, do not forget to include the constraint that the sum of the weights of the stocks in the portfolio should be equal to 1, and the weights should be non-negative.

What is the **optimal portfolio, the optimal expected return, and the risk of the optimal portfolio** (Hint: This problem is an implementation of "Return Maximization Formulation" that you have seen in the class)

In [22]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row.
rate_of_return_df.dropna(inplace = True)

# Calculate the average rate of return for each stock.
RoR_Avg = rate_of_return_df.mean()

# Calculate the covariance between each pair of stocks.
RoR_Cov = rate_of_return_df.cov()

###################################################################
# Create a new model for Question 4: return maximization with risk constraint
model = gb.Model('Question4')
model.Params.LogToConsole = 0
m = len(Stock_Name)



# Decision variables: portfolio weights
x = model.addVars(m, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="x")

# Linear objective: maximize expected return mu' x
mu = RoR_Avg.values
ret_expr = gb.quicksum(mu[i] * x[i] for i in range(m))
model.setObjective(ret_expr, GRB.MAXIMIZE)

# Risk constraint: variance <= (0.04)^2
Sigma = RoR_Cov.values
risk_var_expr = gb.QuadExpr()
for i in range(m):
    for j in range(m):
        risk_var_expr.add(x[i] * Sigma[i, j] * x[j])

model.addQConstr(risk_var_expr <= 0.04**2, name="risk_limit")

# Full investment
model.addConstr(gb.quicksum(x[i] for i in range(m)) == 1.0, name="budget")

# Optimize
model.optimize()

# Extract optimal weights
opt_weights_q4 = [x[i].X for i in range(m)]
w_vec_q4 = np.array(opt_weights_q4)

# Compute portfolio variance and risk (std dev)
port_var_q4 = float(w_vec_q4 @ Sigma @ w_vec_q4)
import math
port_risk_q4 = math.sqrt(port_var_q4)

# Compute portfolio expected return
port_ret_q4 = float(mu @ w_vec_q4)

print("Question 4 – Optimal weights (return maximization with risk ≤ 0.04):")
for i in range(m):
    print(f"  {Stock_Name[i]}: {opt_weights_q4[i]:.6f}")

print(f"\nQuestion 4 – Optimal portfolio expected return: {port_ret_q4:.6f}")
print(f"Question 4 – Portfolio risk (std dev): {port_risk_q4:.6f}")


Set parameter LogToConsole to value 0
Question 4 – Optimal weights (return maximization with risk ≤ 0.04):
  FTSE 100: 0.000000
  DAX: 0.000000
  DJIA: 0.999993
  DJ Asian Titans 50: 0.000000
  Russell 2000: 0.000007

Question 4 – Optimal portfolio expected return: 0.006931
Question 4 – Portfolio risk (std dev): 0.032890


## Question 5: (10 points)

We want to implement one of the practical extensions of the Markowitz problem discussed in class. In particular, we want to implement **the transaction costs**. As a portfolio manager, you rarely build a portfolio from scratch. Instead, you start with an existing portfolio and make some changes. For example, you can sell some of the stocks in your portfolio and buy others. In this case, you have to pay some transaction costs. Here, we assume you are given a portfolio with 20% of your money invested in each of the five indices. You may want to change this portfolio to a new portfolio, but you have to pay the transaction costs.

Here, we want to maximize the expected return minus the transaction costs such that the risk of the new portfolio is less than or equal to 0.04. Therefore, the objective function is:
$$\sum_{i = 1}^5 R_i \times x_i - 0.1 \times \sum_{i = 1}^5 (x_i - 0.2)^2,$$
Where $x_i$ is the weight of the *i*th index in the new portfolio, and $R_i$ is the average return of the *i*th index. Report **the optimal portfolio, the optimal expected return, and the risk of the optimal portfolio.**

In [23]:
StocksData = pd.read_excel('Markowitz.xlsx', header=0, index_col = 0,
                           sheet_name = 'Markowitz', skiprows = 2, nrows= 68, usecols = range(0,6))

Stock_Name = ['FTSE 100', 'DAX', 'DJIA', 'DJ Asian Titans 50', 'Russell 2000']

# Calculate the rate of returns (RoRs).
rate_of_return_df = StocksData.pct_change(-1)
# Drop the earliest row.
rate_of_return_df.dropna(inplace = True)

# Calculate the average rate of return for each stock.
RoR_Avg = rate_of_return_df.mean()

# Calculate the covariance between each pair of stocks.
RoR_Cov = rate_of_return_df.cov()

###################################################################
# Create a new model for Question 5: return - transaction costs, with risk constraint
model = gb.Model('Question5')
model.Params.LogToConsole = 0
model.Params.NonConvex = 2   # Nonconvex due to negative quadratic term in the objective
m = len(Stock_Name)

# Your code here
# Decision variables: portfolio weights
x = model.addVars(m, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="x")

mu = RoR_Avg.values
Sigma = RoR_Cov.values

# Expected return term: sum_i R_i * x_i
ret_expr = gb.quicksum(mu[i] * x[i] for i in range(m))

# Transaction cost term: 0.1 * sum_i (x_i - 0.2)^2
trans_cost_expr = gb.QuadExpr()
for i in range(m):
    trans_cost_expr.add((x[i] - 0.2) * (x[i] - 0.2))

# Full objective: maximize expected return minus transaction cost
model.setObjective(ret_expr - 0.1 * trans_cost_expr, GRB.MAXIMIZE)

# Risk constraint: variance <= (0.04)^2
risk_var_expr = gb.QuadExpr()
for i in range(m):
    for j in range(m):
        risk_var_expr.add(x[i] * Sigma[i, j] * x[j])

model.addQConstr(risk_var_expr <= 0.04**2, name="risk_limit")

# Full investment
model.addConstr(gb.quicksum(x[i] for i in range(m)) == 1.0, name="budget")

# Optimize
model.optimize()

# Extract optimal weights
opt_weights_q5 = [x[i].X for i in range(m)]
w_vec_q5 = np.array(opt_weights_q5)

# Compute portfolio variance and risk
import math
port_var_q5 = float(w_vec_q5 @ Sigma @ w_vec_q5)
port_risk_q5 = math.sqrt(port_var_q5)

# Compute portfolio expected return
port_ret_q5 = float(mu @ w_vec_q5)

print("Question 5 – Optimal weights (with transaction costs):")
for i in range(m):
    print(f"  {Stock_Name[i]}: {opt_weights_q5[i]:.6f}")

print(f"\nQuestion 5 – Optimal portfolio expected return: {port_ret_q5:.6f}")
print(f"Question 5 – Portfolio risk (std dev): {port_risk_q5:.6f}")


Set parameter LogToConsole to value 0
Question 5 – Optimal weights (with transaction costs):
  FTSE 100: 0.183742
  DAX: 0.212041
  DJIA: 0.214827
  DJ Asian Titans 50: 0.174954
  Russell 2000: 0.214436

Question 5 – Optimal portfolio expected return: 0.004260
Question 5 – Portfolio risk (std dev): 0.037031


## Question 6: (5 points)

In class, you saw that it is necessary to set *model.params.NonConvex = 2* to solve some nonlinear problems. However, this was not the case for Problem 1 in this assignment. Why?

*Your answer here*

## Question 6

**Answer (for the markdown cell after Q6):**

For Problem 1, all models (Questions 3 and 4) have **convex quadratic structures**:

- The covariance matrix Σ is **positive semidefinite**, so the variance term $ x^\top \Sigma x $ is convex.
- In Question 3 we **minimize** a convex quadratic objective over linear constraints.
- In Question 4 we **maximize** a linear objective over a region defined by a convex quadratic constraint and linear constraints.

These are standard **convex QP/QCQP problems**, which Gurobi solves directly without setting  
`model.Params.NonConvex = 2`.

`NonConvex = 2` is required **only** when the model contains nonconvex quadratic or bilinear terms (like in Question 5, where the objective has a **negative** quadratic penalty term).

# Problem 2: (50 points)

## The Story and Data

John Brooks, managing director of the CYCOM Corporation, is trying to determine how to allocate CYCOM's research and development budget for the coming year. Six different projects are under consideration. John believes that the success of every project depends in part on the number of engineers assigned to the project. Let $x_1, ..., x_6$ denote the number of engineers assigned to projects 1,...,6. According to internal corporate estimates, the probability that each project will be successful is given according to the function in the third column of Table 1:


| Project | Start-up Cost (in \$1000) | Probability of Success | Profit if Successful (in \$1000) |
| --- | --- | --- | --- |
| 1 | 325 | $\frac{x_1}{x_1 + 1.1}$ | 1,750 |
| 2 | 200 | $\frac{x_2}{x_2 + 0.5}$ | 700 |
| 3 | 490 | $\frac{x_3}{x_3 + 2.5}$ | 1,300 |
| 4 | 90 | $\frac{x_4}{x_4 + 1.6}$ | 800 |
| 5 | 210 | $\frac{x_5}{x_5 + 2.2}$ | 1,450 |
| 6 | 150 | $\frac{x_6}{x_6 + 2.4}$ | 1,300 |


For example, if 10 engineers are assigned to project 1, the probability of success in that project is $\frac{10}{10 + 1.1} = 0.901$. Cycom can assign up to 25 engineers to these six projects. The cost of an engineer is $\$150,000$ (per year). An engineer can be assigned to more than one project. For example, $x_2$ = 3.4 means that 3.4 engineers are assigned to Project 2, which means that 3 engineers are assigned to this project full-time, and one engineer is assigned to this project only 40% of the time. Nonetheless, the total number of hired engineers should be an integer less than or equal to 25.

If a project is unsuccessful, it incurs its full start-up cost, shown in the second column of Table 1. If the project is successful, it realizes the profit value shown in the fourth column of Table 1. This value includes the start-up cost of the project (but does not include the cost of engineers). Thus, for example, if Project 1 is unsuccessful, it incurs a cost of $\$325,000$. If Project 1 is successful, it will realize a profit of $\$1,750,000$.

In this problem, we assume that the manager does not have the option of abandoning a project to prevent its start-up cost. The only thing the manager can choose is the number of engineers assigned to each project.

## Question 7: The Expected Profit Approach

As a business, CYCOM wants to maximize its total expected profit. The total expected profit is the sum of the expected profit of each project minus the total cost of the engineers. The expected profit of each project is the probability of its success times the profit if successful minus the probability of its failure multiplied by the start-up cost. For instance, suppose the probability of success for Project 1 is $P_1$. Then, the expected profit for Project 1 is $P_1 \times 1,750,000 - (1 - P_1) \times 325,000$.

### Question 7.1: The Formulation (10 points)

Complete the formulation of the problem of maximizing the total expected profit of CYCOM. You may use the following values as given: \
$b_i$: profit if project $i$ is successful. \
$c_i$: start-up cost of project $i$. \
$a_i$: the constant term in the denominator of the probability of success of project $i$.

### Decision Variables:
$x_i$: The number of engineers assigned to project $i$ - non-negative continuous variables. \
$y$: Total number of engineers hired - non-negative integer variable.

### Objective Function:
... look at the next block please

### Constraints:
... look at the next block please

## Question 7.1 – Expected profit formulation


### Decision variables:
- $x_i \geq 0$: Number of engineers (FTEs) assigned to project $i = 1, \dots, 6$
- $y \in \mathbb{Z}_+$: Total number of engineers hired
- (Optional modeling variable for implementation) $P_i \in [0,1]$: probability of success of project $i$,  
  with $P_i = \dfrac{x_i}{x_i + a_i}$.

### Objective (maximize total expected profit, in $1000):
$$
\max \sum_{i=1}^{6} \left( P_i b_i - (1-P_i) c_i \right) - 150 y.
$$

### Constraints:

1. **Probability-assignment relation for each project:**
   $$
   P_i = \dfrac{x_i}{x_i + a_i}, \quad i = 1, \dots, 6.
   $$

2. **Total assigned FTEs cannot exceed total engineers hired:**
   $$
   \sum_{i=1}^{6} x_i \leq y.
   $$

3. **Upper bound on engineers:**
   $$
   y \leq 25.
   $$

4. **Nonnegativity and integrality:**
   $$
   x_i \geq 0, \quad P_i \in [0,1], \quad y \in \mathbb{Z}_+.
   $$

In the code we enforce $P_i = x_i/(x_i + a_i)$ using bilinear constraints.

### Question 7.2: The Code (10 points)

Implement the formulation you introduced in Question 7.1 in Python. Then, **find and report the optimal total expected profit of CYCOM, total number of engineers, and the number of engineers assigned to each project.**

**Technical Note 1:**  Unlike solving linear programs with Gurobi, where **scaling** did not matter that much, in nonlinear programs, the scale matters. For instance, if this were a linear program, using $\$150,000$ or $\$150$ as the cost of an engineer would not make any difference. However, in NLPs, using $\$150,000$ instead of $\$150$ can make the problem much more complex for Gurobi. Therefore, I recommend using the costs in thousands of dollars instead of dollars.

**Technical Note 2:** Gurobi does not allow you to introduce constraints/objective functions in fractional format. Think about how you can convert the fractional constraints/objective function to linear constraints/objective function. Maybe introducing a new variable $p_i$ and setting $p_i . (x_i + a_i) = x_i$ can help you.

In [24]:
SetupCost = [325, 200, 490, 125, 710, 240]   # c_i in $1000
Profit    = [1750, 700, 1300, 800, 1450, 1300]  # b_i in $1000
Denom     = [1.1, 0.5, 2.5, 1.6, 2.2, 2.4]   # a_i
n = len(Profit)

engineer_cost = 150.0   # in $1000 per engineer
max_engineers = 25

# Create a new model.
model = gb.Model('Question7')
model.Params.LogToConsole = 0
model.Params.NonConvex = 2    # required for bilinear constraints


# Your code here

# Decision variables:
# x[i] = number of engineer FTE assigned to project i
x = model.addVars(n, lb=0.0, vtype=GRB.CONTINUOUS, name="x")

# y = integer number of engineers hired
y = model.addVar(lb=0.0, ub=max_engineers, vtype=GRB.INTEGER, name="y")

# p[i] = probability of success of project i, in [0,1]
p = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="p")

# Link p[i] and x[i] via p_i = x_i / (x_i + a_i):
# p_i * (x_i + a_i) = x_i  →  p_i * x_i + a_i * p_i - x_i = 0  (bilinear)
for i in range(n):
    model.addConstr(p[i] * (x[i] + Denom[i]) == x[i],
                    name=f"prob_def_{i+1}")

# Total FTE assigned cannot exceed total engineers hired
model.addConstr(gb.quicksum(x[i] for i in range(n)) <= y,
                name="fte_le_y")

# Upper bound on engineers
model.addConstr(y <= max_engineers, name="max_eng")

# Objective: maximize total expected profit minus engineer costs
# Expected project profit: p_i * b_i - (1 - p_i) * c_i
expected_profit_expr = gb.quicksum(
    p[i] * Profit[i] - (1.0 - p[i]) * SetupCost[i]
    for i in range(n)
)

# Total expected CYCOM profit (in $1000)
total_expected_profit = expected_profit_expr - engineer_cost * y

model.setObjective(total_expected_profit, GRB.MAXIMIZE)

# Optimize
model.optimize()

# Extract optimal solution
opt_x_q7 = [x[i].X for i in range(n)]
opt_y_q7 = y.X
opt_p_q7 = [p[i].X for i in range(n)]
opt_exp_profit_q7 = total_expected_profit.getValue()

print("Question 7 – Expected profit maximization")
print(f"Optimal number of engineers hired (y): {opt_y_q7:.0f}")
for i in range(n):
    print(f"  Project {i+1}: x = {opt_x_q7[i]:.4f} FTE, p = {opt_p_q7[i]:.4f}")

print(f"\nQuestion 7 – Optimal expected total profit (in $1000): {opt_exp_profit_q7:.2f}")
print(f"Question 7 – Optimal expected total profit (in $): {opt_exp_profit_q7*1000:.2f}")


Set parameter LogToConsole to value 0
Question 7 – Expected profit maximization
Optimal number of engineers hired (y): 15
  Project 1: x = 2.8733 FTE, p = 0.7232
  Project 2: x = 1.2655 FTE, p = 0.7168
  Project 3: x = 3.0661 FTE, p = 0.5508
  Project 4: x = 1.6019 FTE, p = 0.5003
  Project 5: x = 3.5343 FTE, p = 0.6163
  Project 6: x = 2.6588 FTE, p = 0.5256

Question 7 – Optimal expected total profit (in $1000): 1395.17
Question 7 – Optimal expected total profit (in $): 1395165.77


## Question 8: Minimizing the Uncertainty Approach

Similar to the stock market problem, we can also use the uncertainty approach to solve this problem. In this approach, we want to minimize the uncertainty (the standard deviation) in the total profit of CYCOM. For this, suppose the success or failure of each project is independent of the success or failure of other projects. Then, the variance of the total expected profit of CYCOM is the sum of the variances of the expected profits of each project. Although we want to decrease the uncertainty in the total profit of CYCOM, we want the expected profit of CYCOM to be at least $1.1 million.

**N.B.:** If a random variable gets value $A$ with probability $p$ and value $B$ with probability $1 - p$, then the variance of that random variable is $(A - B)^2 \times p \times (1 - p)$.

### Question 8.1: The Formulation (15 points)

Complete the formulation of the problem of minimizing the uncertainty in the total profit of CYCOM. You may use the following values as given: \
$b_i$: profit if project $i$ is successful. \
$c_i$: start-up cost of project $i$. (A number greater than zero.) \
$a_i$: the constant term in the denominator of the probability of success of project $i$.

### Decision Variables:
$x_i$: The number of engineers assigned to project $i$ - non-negative continuous variables. \
$y$: Total number of engineers hired - non-negative integer variable.

### Objective Function:
...

### Constraints:
...

## Question 8.1 – Uncertainty minimization formulation (markdown)


### Decision variables:
- $x_i \geq 0$: number of engineer FTEs assigned to project $i$
- $y \in \mathbb{Z}_+$: total number of engineers hired ($y \leq 25$)
- $P_i \in [0,1]$: probability of success of project $i$, $P_i = \dfrac{x_i}{x_i + a_i}$.

### Expected total profit (in $1000):
$$
\mathrm{E[profit]} = \sum_{i=1}^{6} \bigl( P_i b_i - (1-P_i) c_i \bigr) - 150 y.
$$

### Variance of total profit (uncertainty measure, in ($1000)^2):

since project outcomes are independent, and project $i$ yields profit $b_i$ or $-c_i$:

$$
\mathrm{Var}(i) = (b_i - (-c_i))^2 P_i (1-P_i) = (b_i + c_i)^2 P_i (1-P_i),
$$
$$
\mathrm{Var(total\ profit)} = \sum_{i=1}^{6} (b_i + c_i)^2 P_i (1-P_i).
$$

### Objective:
$$
\min \sum_{i=1}^{6} (b_i + c_i)^2 P_i (1-P_i).
$$

### Constraints:

1. **Probability-assignment relation (for all i):**
   $$
   P_i = \dfrac{x_i}{x_i + a_i}.
   $$

2. **Expected profit at least $1.1M:**
   $$
   \sum_{i=1}^{6} \bigl( P_i b_i - (1-P_i) c_i \bigr) - 150 y \geq 1100.
   $$

3. **Total FTE not exceeding hired engineers:**
   $$
   \sum_{i=1}^{6} x_i \leq y.
   $$

4. **Engineer limit and domains:**
   $$
   y \leq 25, \quad x_i \geq 0, \quad P_i \in [0,1], \quad y \in \mathbb{Z}_+.
   $$

### Question 8.2: The Code (15 points)

Implement the formulation you introduced in Question 8.1 in Python. Then, **find and report the minimum standard deviation of the total expected profit, the total expected profit, total number of engineers, and the number of engineers assigned to each project.**

In [25]:
SetupCost = [325, 200, 490, 125, 710, 240]   # c_i in $1000
Profit    = [1750, 700, 1300, 800, 1450, 1300]  # b_i in $1000
Denom     = [1.1, 0.5, 2.5, 1.6, 2.2, 2.4]   # a_i
n = len(Profit)

engineer_cost = 150.0    # in $1000 per engineer
max_engineers = 25
min_expected_profit = 1100.0   # in $1000 ( = $1.1M )

# Create a new model.
model = gb.Model('Question8')
model.Params.LogToConsole = 0
model.Params.NonConvex = 2      # required: bilinear + p_i(1-p_i)


# Your code here
# Decision variables:
x = model.addVars(n, lb=0.0, vtype=GRB.CONTINUOUS, name="x")            # FTE per project
y = model.addVar(lb=0.0, ub=max_engineers, vtype=GRB.INTEGER, name="y") # # engineers
p = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="p")    # success prob

# Probability definition: p_i = x_i / (x_i + a_i) via bilinear equality
for i in range(n):
    model.addConstr(p[i] * (x[i] + Denom[i]) == x[i], name=f"prob_def_{i+1}")

# Total FTE ≤ total engineers
model.addConstr(gb.quicksum(x[i] for i in range(n)) <= y, name="fte_le_y")

# Engineer upper bound
model.addConstr(y <= max_engineers, name="max_eng")

# Expected profit expression (in $1000)
expected_profit_expr = gb.quicksum(
    p[i] * Profit[i] - (1.0 - p[i]) * SetupCost[i]
    for i in range(n)
) - engineer_cost * y

# Expected profit constraint: at least $1.1M
model.addConstr(expected_profit_expr >= min_expected_profit,
                name="min_expected_profit")

# Objective: minimize variance of total profit
# Var(total) = sum_i (b_i + c_i)^2 * p_i * (1 - p_i)
variance_expr = gb.QuadExpr()
for i in range(n):
    factor = (Profit[i] + SetupCost[i]) ** 2
    variance_expr.add(factor * p[i] * (1.0 - p[i]))

model.setObjective(variance_expr, GRB.MINIMIZE)

# Optimize
model.optimize()

# Extract optimal solution
opt_x_q8 = [x[i].X for i in range(n)]
opt_y_q8 = y.X
opt_p_q8 = [p[i].X for i in range(n)]
opt_var_q8 = variance_expr.getValue()
opt_exp_profit_q8 = expected_profit_expr.getValue()

print("Question 8 – Uncertainty (variance) minimization with expected profit ≥ $1.1M")
print(f"Optimal number of engineers hired (y): {opt_y_q8:.0f}")
for i in range(n):
    print(f"  Project {i+1}: x = {opt_x_q8[i]:.4f} FTE, p = {opt_p_q8[i]:.4f}")

print(f"\nQuestion 8 – Minimum variance of total profit (in (1000$)^2): {opt_var_q8:.2f}")
print(f"Question 8 – Corresponding expected profit (in $1000): {opt_exp_profit_q8:.2f}")
print(f"Question 8 – Corresponding expected profit (in $): {opt_exp_profit_q8*1000:.2f}")


Set parameter LogToConsole to value 0
Question 8 – Uncertainty (variance) minimization with expected profit ≥ $1.1M
Optimal number of engineers hired (y): 21
  Project 1: x = 5.0895 FTE, p = 0.8223
  Project 2: x = 1.6348 FTE, p = 0.7658
  Project 3: x = 4.1356 FTE, p = 0.6232
  Project 4: x = 1.2972 FTE, p = 0.4477
  Project 5: x = 6.0115 FTE, p = 0.7321
  Project 6: x = 2.8314 FTE, p = 0.5412

Question 8 – Minimum variance of total profit (in (1000$)^2): 3242377.18
Question 8 – Corresponding expected profit (in $1000): 1100.00
Question 8 – Corresponding expected profit (in $): 1100000.00
